In [2]:
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import csv
import helpers
from numpy import linalg as LA

In [13]:
## load train data 
y_tr, tx_tr, ids_tr = helpers.load_csv_data('train.csv', sub_sample = False)
y_te, tx_te, ids_te = helpers.load_csv_data('test.csv', sub_sample = False)

In [7]:
## implementation du classifier SVM_UltraBoost
## son train prend un peu de temps, il se fait en cinq étapes, avec un print 'ok' 
## à chacune d'entres elles
## faire tourner toutes les fonctions plus bas avant de faire tourner cette partie
## le préprocessing se fait en interne, donc pas besoin de le faire ici

## N.B. plus on augmente le paramètre size, plus le temps de train est long
## il s'agit du nombre de points sur lequel est entrainé chaque "sous-classifier"


classifier = SVM_UltraBoost(learning_rate=0.001, lambda_param=0.01, n_iters=1000, size = 500)
classifier.fit(tx_tr, y_tr)

/var/folders/ml/_g1gzlvn3dx4lm6xz6kbjghm0000gn/T/ipykernel_1727/4212815220.py:388: RuntimeWarning: Mean of empty slice.
  if np.isnan(column[column!=-999.].mean())==False:


ok1
ok2
ok3
ok4
ok5


In [14]:
pred = classifier.predict(tx_te)
print(accuracy(y_tr, pred))

/var/folders/ml/_g1gzlvn3dx4lm6xz6kbjghm0000gn/T/ipykernel_1727/4212815220.py:388: RuntimeWarning: Mean of empty slice.
  if np.isnan(column[column!=-999.].mean())==False:


0.565236


In [27]:
length = len(pred)
export_to_csv(np.reshape(pred, (length, 1)), np.reshape(ids_te, (length, 1)), 'submissions2')

In [25]:
def export_to_csv(ypred,Id_test, text):
    output = np.concatenate((Id_test, ypred),axis = 1)
    np.savetxt(text, output, delimiter=",",fmt= "%i", header = "Id,Prediction")
    return None

In [5]:
## every useful classes : 
## SVM_UltraBoost
## SVM_boosted3
## SVM_boosted2
## SVM_boosted
## SVM

class SVM_UltraBoost:
    
    ## train 5 identical SVM_boosted3 models, on different subsets of the train data
    ## then return the most occurent prediction of the 5 classifiers  
    
    def __init__(self, learning_rate=0.001, lambda_param=0.01, n_iters=1000, size = 500):
        self.lr = learning_rate
        self.lambda_param = lambda_param
        self.n_iters = n_iters
        self.size = size
        self.SVM1 = SVM_boosted3(self.lr, self.lambda_param, self.n_iters, self.size)
        self.SVM2 = SVM_boosted3(self.lr, self.lambda_param, self.n_iters, self.size)
        self.SVM3 = SVM_boosted3(self.lr, self.lambda_param, self.n_iters, self.size)
        self.SVM4 = SVM_boosted3(self.lr, self.lambda_param, self.n_iters, self.size)
        self.SVM5 = SVM_boosted3(self.lr, self.lambda_param, self.n_iters, self.size)
        
    def fit(self, X, Y):

        """Computes the optimal model parameters w and b for the soft vector model 
        Args:
        y: shape=(N, 1)
        X: shape=(N,M)
        w: shape=(M, 1). The vector of model parameters.
        lambda_ : Ridge regularization parameter
        Returns:
        Value of optimal w  
        """
        length = X.shape[0]
        intervals = np.linspace(0, length, 6).astype(int)
        
        ## we create the classifiers
        self.SVM1.fit(X[intervals[0]:intervals[1]], Y[intervals[0]:intervals[1]].flatten())
        print('ok1')
        self.SVM2.fit(X[intervals[1]:intervals[2]], Y[intervals[1]:intervals[2]].flatten())
        print('ok2')
        self.SVM3.fit(X[intervals[2]:intervals[3]], Y[intervals[2]:intervals[3]].flatten())
        print('ok3')
        self.SVM4.fit(X[intervals[3]:intervals[4]], Y[intervals[3]:intervals[4]].flatten())
        print('ok4')
        self.SVM5.fit(X[intervals[4]:intervals[5]], Y[intervals[4]:intervals[5]].flatten())
        print('ok5')
        
        
    def predict(self, X):
        """Predictions with new data on SVM 
        """
        
        ## we make the predictions
        Y1 = self.SVM1.predict(X)
        Y2 = self.SVM1.predict(X)
        Y3 = self.SVM1.predict(X)
        Y4 = self.SVM1.predict(X)
        Y5 = self.SVM1.predict(X)
        
        Y = (Y1 + Y2 + Y3 + Y4 + Y5)
        Y[Y >= 0.] =  1.
        Y[Y <  0.] = -1.
        
        return Y

class SVM_boosted3:
    
    def __init__(self, learning_rate=0.001, lambda_param=0.01, n_iters=1000, size = 1000):
        self.lr = learning_rate
        self.lambda_param = lambda_param
        self.n_iters = n_iters
        self.size = size
        self.SVM1 = SVM_boosted2(self.lr, self.lambda_param, self.n_iters, self.size)
        self.SVM2 = SVM_boosted2(self.lr, self.lambda_param, self.n_iters, self.size)
        
    def fit(self, X, Y):

        """Computes the optimal model parameters w and b for the soft vector model 
        Args:
        y: shape=(N, 1)
        X: shape=(N,M)
        w: shape=(M, 1). The vector of model parameters.
        lambda_ : Ridge regularization parameter
        Returns:
        Value of optimal w  
        """
        
        ## we prepare all subsets
        S1 = (X[:, 27] >= 0.)
        S2 = (X[:, 27] <  0.)
        
        ## we create the classifiers
        self.SVM1.fit(X[S1], Y[S1].flatten())
        self.SVM2.fit(X[S2], Y[S2].flatten())
        
        
    def predict(self, X):
        """Predictions with new data on SVM 
        """
        
        ## we prepare all the subsets
        S1 = (X[:, 27] >= 0.)
        S2 = (X[:, 27] <  0.)
        
        ## we make the predictions
        Y = np.ones(X.shape[0])
        Y[S1] = self.SVM1.predict(X[S1])
        Y[S2] = self.SVM2.predict(X[S2])
        
        return Y

class SVM_boosted2:
    
    def __init__(self, learning_rate=0.001, lambda_param=0.01, n_iters=1000, size = 1000):
        self.lr = learning_rate
        self.lambda_param = lambda_param
        self.n_iters = n_iters
        self.size = size
        self.SVM1 = SVM_boosted(self.lr, self.lambda_param, self.n_iters, self.size)
        self.SVM2 = SVM_boosted(self.lr, self.lambda_param, self.n_iters, self.size)
        self.SVM3 = SVM_boosted(self.lr, self.lambda_param, self.n_iters, self.size)
        self.SVM4 = SVM_boosted(self.lr, self.lambda_param, self.n_iters, self.size)
        
    def fit(self, X, Y):

        """Computes the optimal model parameters w and b for the soft vector model 
        Args:
        y: shape=(N, 1)
        X: shape=(N,M)
        w: shape=(M, 1). The vector of model parameters.
        lambda_ : Ridge regularization parameter
        Returns:
        Value of optimal w  
        """
        
        ## we prepare all subsets
        S1 = (X[:, 22]==0.)
        S2 = (X[:, 22]==1.)
        S3 = (X[:, 22]==2.)
        S4 = (X[:, 22]==3.)
        
        
        ## we create the classifiers
        self.SVM1.fit(X[S1], Y[S1].flatten())
        self.SVM2.fit(X[S2], Y[S2].flatten())
        self.SVM3.fit(X[S3], Y[S3].flatten())
        self.SVM4.fit(X[S4], Y[S4].flatten())
        
    def predict(self, X):
        """Predictions with new data on SVM 
        """
        
        ## we prepare all the subsets
        S1 = (X[:, 22]==0.)
        S2 = (X[:, 22]==1.)
        S3 = (X[:, 22]==2.)
        S4 = (X[:, 22]==3.)
        
        ## we make the predictions
        Y = np.ones(X.shape[0])
        Y[S1] = self.SVM1.predict(X[S1])
        Y[S2] = self.SVM2.predict(X[S2])
        Y[S3] = self.SVM3.predict(X[S3])
        Y[S4] = self.SVM4.predict(X[S4])
        
        
        return Y

class SVM_boosted:

    def __init__(self, learning_rate=0.001, lambda_param=0.01, n_iters=1000, size = 1000):
        self.lr = learning_rate
        self.lambda_param = lambda_param
        self.n_iters = n_iters
        self.size = size
        self.SVM1 = SVM(self.lr, self.lambda_param, self.n_iters)
        self.SVM2 = SVM(self.lr, self.lambda_param, self.n_iters)
        self.SVM3 = SVM(self.lr, self.lambda_param, self.n_iters)
        self.SVM4 = SVM(self.lr, self.lambda_param, self.n_iters)
        self.SVM5 = SVM(self.lr, self.lambda_param, self.n_iters)
        self.SVM6 = SVM(self.lr, self.lambda_param, self.n_iters)
        self.SVM7 = SVM(self.lr, self.lambda_param, self.n_iters)
        self.SVM8 = SVM(self.lr, self.lambda_param, self.n_iters)
        self.SVM9 = SVM(self.lr, self.lambda_param, self.n_iters) 
        
    def fit(self, X, Y):

        """Computes the optimal model parameters w and b for the soft vector model 
        Args:
        y: shape=(N, 1)
        X: shape=(N,M)
        w: shape=(M, 1). The vector of model parameters.
        lambda_ : Ridge regularization parameter
        Returns:
        Value of optimal w  
        """
        
        ## we prepare all subsets
        S1 = (X[:, 0]!=-999.)&(X[:, 23]!=-999.)&(X[:, 4]!=-999.)&(X[:,0]>118.)
        S2 = (X[:, 0]!=-999.)&(X[:, 23]!=-999.)&(X[:, 4]!=-999.)&(X[:,0]<=118.)
        S3 = (X[:, 0]!=-999.)&(X[:, 23]==-999.)&(X[:, 4]==-999.)&(X[:,0]>118.)
        S4 = (X[:, 0]!=-999.)&(X[:, 23]==-999.)&(X[:, 4]==-999.)&(X[:,0]<=118.)
        S5 = (X[:, 0]!=-999.)&(X[:, 23]!=-999.)&(X[:, 4]==-999.)&(X[:,0]>118.)
        S6 = (X[:, 0]!=-999.)&(X[:, 23]!=-999.)&(X[:, 4]==-999.)&(X[:,0]<=118.)
        S7 = (X[:, 0]==-999.)&(X[:, 23]!=-999.)&(X[:, 4]!=-999.)
        S8 = (X[:, 0]==-999.)&(X[:, 23]!=-999.)&(X[:, 4]==-999.)
        S9 = (X[:, 0]==-999.)&(X[:, 23]==-999.)&(X[:, 4]==-999.)
        
        ## we create the classifiers
        self.SVM1.fit(remove_unique(standardize(treat_data(X[S1][:self.size]))), Y[S1][:self.size].flatten())
        self.SVM2.fit(remove_unique(standardize(treat_data(X[S2][:self.size]))), Y[S2][:self.size].flatten())
        self.SVM3.fit(remove_unique(standardize(treat_data(X[S3][:self.size]))), Y[S3][:self.size].flatten())
        self.SVM4.fit(remove_unique(standardize(treat_data(X[S4][:self.size]))), Y[S4][:self.size].flatten())
        self.SVM5.fit(remove_unique(standardize(treat_data(X[S5][:self.size]))), Y[S5][:self.size].flatten())
        self.SVM6.fit(remove_unique(standardize(treat_data(X[S6][:self.size]))), Y[S6][:self.size].flatten())
        self.SVM7.fit(remove_unique(standardize(treat_data(X[S7][:self.size]))), Y[S7][:self.size].flatten())
        self.SVM8.fit(remove_unique(standardize(treat_data(X[S8][:self.size]))), Y[S8][:self.size].flatten())
        self.SVM9.fit(remove_unique(standardize(treat_data(X[S9][:self.size]))), Y[S9][:self.size].flatten())
        
    def predict(self, X):
        """Predictions with new data on SVM 
        """
        
        ## we prepare all the subsets
        S1 = (X[:, 0]!=-999.)&(X[:, 23]!=-999.)&(X[:, 4]!=-999.)&(X[:,0]>118.)
        S2 = (X[:, 0]!=-999.)&(X[:, 23]!=-999.)&(X[:, 4]!=-999.)&(X[:,0]<=118.)
        S3 = (X[:, 0]!=-999.)&(X[:, 23]==-999.)&(X[:, 4]==-999.)&(X[:,0]>118.)
        S4 = (X[:, 0]!=-999.)&(X[:, 23]==-999.)&(X[:, 4]==-999.)&(X[:,0]<=118.)
        S5 = (X[:, 0]!=-999.)&(X[:, 23]!=-999.)&(X[:, 4]==-999.)&(X[:,0]>118.)
        S6 = (X[:, 0]!=-999.)&(X[:, 23]!=-999.)&(X[:, 4]==-999.)&(X[:,0]<=118.)
        S7 = (X[:, 0]==-999.)&(X[:, 23]!=-999.)&(X[:, 4]!=-999.)
        S8 = (X[:, 0]==-999.)&(X[:, 23]!=-999.)&(X[:, 4]==-999.)
        S9 = (X[:, 0]==-999.)&(X[:, 23]==-999.)&(X[:, 4]==-999.)
        
        ## we make the predictions
        Y = np.ones(X.shape[0])
        Y[S1] = self.SVM1.predict(remove_unique(standardize(treat_data(X[S1]))))
        Y[S2] = self.SVM2.predict(remove_unique(standardize(treat_data(X[S2]))))
        Y[S3] = self.SVM3.predict(remove_unique(standardize(treat_data(X[S3]))))
        Y[S4] = self.SVM4.predict(remove_unique(standardize(treat_data(X[S4]))))
        Y[S5] = self.SVM5.predict(remove_unique(standardize(treat_data(X[S5]))))
        Y[S6] = self.SVM6.predict(remove_unique(standardize(treat_data(X[S6]))))
        Y[S7] = self.SVM7.predict(remove_unique(standardize(treat_data(X[S7]))))
        Y[S8] = self.SVM8.predict(remove_unique(standardize(treat_data(X[S8]))))
        Y[S9] = self.SVM9.predict(remove_unique(standardize(treat_data(X[S9]))))
        
        return Y
        
    def standardize(x):
        centered_data = x - np.mean(x, axis=0)
        std = np.std(centered_data, axis=0)
        std[std==0.] = 1
        std_data = centered_data / std
        return std_data
    
    def treat_data(x, mean = True):
    
        '''
        get an imput array of features (columns) and replace all -999. values 
        by the mean value of their column.
    
        '''
    
        xx = x.copy()
        for ind, column in enumerate(xx.T[:]):
            if mean:
                if np.isnan(column[column!=-999.].mean())==False:
                    column[column==-999.] = column[column!=-999.].mean()
                else: 
                    column[column==-999.] = 0
            
            else:
                return
            ##rien pour l'instant
            xx.T[ind] = column
        
        return xx
    
    def remove_unique(xx):
        xxx = np.array(xx.copy())
        liste = []
        for ind, column in enumerate(xx.T[:]):
            if np.std(column)==0.:
                liste.append(ind)
        xxx = np.delete(xxx, obj = liste  , axis = 1) 
        return xxx
    

class SVM:

    def __init__(self, learning_rate=0.001, lambda_param=0.01, n_iters=1000):
        self.lr = learning_rate
        self.lambda_param = lambda_param
        self.n_iters = n_iters
        self.w = None
        self.b = None


    def fit(self, X, y):

        """Computes the optimal model parameters w and b for the soft vector model 
        Args:
        y: shape=(N, 1)
        X: shape=(N,M)
        w: shape=(M, 1). The vector of model parameters.
        lambda_ : Ridge regularization parameter
        Returns:
        Value of optimal w  
        """
        n_samples, n_features = X.shape
        
        y_ = np.where(y <= 0, -1, 1)
        self.w = np.zeros(n_features)
        self.b = 0

        for _ in range(self.n_iters):
            for idx, x_i in enumerate(X):
                condition = y_[idx] * (np.dot(x_i, self.w) - self.b) >= 1
                if condition:
                    self.w -= self.lr * (2 * self.lambda_param * self.w)
                else:
                    self.w -= self.lr * (2 * self.lambda_param * self.w - np.dot(x_i, y_[idx]))
                    self.b -= self.lr * y_[idx]


    def predict(self, X):
        """Predictions with new data on SVM 
        """
        approx = np.dot(X, self.w) - self.b
        return np.sign(approx)
    
def confusion_matrix(y,ypred):
    """ Evaluates the confusion matrix for classification 
    Args:
        y: shape=(N, 1) variable to be predicted
        ypred: shape=(N, 1) variable predicted by the model
    Returns:
         confusion matrix
    """
    fp = 0
    fn = 0
    tp = 0
    tn = 0
    n=len(y)
    for actual_value, predicted_value in zip(y, ypred):
        if predicted_value == actual_value:
            if predicted_value == 1:
                tp += 1
            else: 
                tn += 1
        else: 
            if predicted_value == 1:
                fp += 1
            else:
                fn += 1
                
    confusion_matrix = [[tn/n, fp/n], [fn/n, tp/n]]
    confusion_matrix = np.array(confusion_matrix)
    return confusion_matrix

def accuracy(y,ypred): 
    matrix = confusion_matrix(y,ypred)
    return matrix[0,0]+ matrix[1,1]

def standardize(x):
        centered_data = x - np.mean(x, axis=0)
        
        std = np.std(centered_data, axis=0)
        std[std==0.] = 1
        
        std_data = centered_data / std
        
    
        return std_data
    
def treat_data(x, mean = True):
    
    '''
    get an imput array of features (columns) and replace all -999. values 
    by the mean value of their column.
    '''
    
    xx = x.copy()
    for ind, column in enumerate(xx.T[:]):
        if mean:
            if np.isnan(column[column!=-999.].mean())==False:
                column[column==-999.] = column[column!=-999.].mean()
            else: 
                column[column==-999.] = 0
            
        else:
            return
            ##rien pour l'instant
        xx.T[ind] = column
        
    return xx

def remove_unique(xx):
    xxx = np.array(xx.copy())
    liste = []
    for ind, column in enumerate(xx.T[:]):
        if np.std(column)==0.:
            liste.append(ind)
    xxx = np.delete(xxx, obj = liste  , axis = 1) 
    return xxx


def log_right_skewed(x):
    eps = 1e-6
    xx = x.copy()
    #ids = np.array([0, 1, 2, 3, 5, 8, 9, 10, 13, 16, 23, 26, 29])
    idx_3 = np.array([0, 1, 2, 5, 8, 9, 10, 13, 19, 23, 26, 29])
    for i in idx_3:
        xxx = xx[xx[:,i]!=-999.] 
        minimum = xxx[:, i].min()
        xx[xx[:,i]!=-999.][:, i] = np.log(1+xxx[:, i]+eps-minimum)
    return xx
    
    